In [16]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split

In [114]:
# example of progressively loading images from file
from keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator()
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory('training', class_mode='binary', target_size=(60, 60))
test_it = datagen.flow_from_directory('test', class_mode='binary', target_size=(60, 60))
# confirm the iterator works
batchX, batchy = train_it.next()
batchXt, batchyt = test_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchXt.shape, batchXt.min(), batchXt.max()))

Found 320 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Batch shape=(32, 60, 60, 3), min=0.000, max=255.000
Batch shape=(32, 60, 60, 3), min=0.000, max=255.000


In [117]:
# Generate a model with all layers (with top)
vgg16 = VGG16(weights='imagenet', include_top=True, input_shape=(60, 60, 3))

#Add a layer where input is the output of the  second last layer 
x = Dense(2, activation='softmax', name='predictions')(vgg16.layers[-2].output)

#Then create the corresponding model 
my_model = Model(inputs=vgg16.input, outputs=x)
my_model.summary()

ValueError: When setting `include_top=True` and loading `imagenet` weights, `input_shape` should be (224, 224, 3).

In [112]:
for i in range(0,len(vgg16.layers)):
    my_model.layers[i].trainable = False

In [113]:
my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
my_model.fit(batchX, batchy, epochs=5, verbose=1)

Epoch 1/5
1/1 [==============================] - 0s 2ms/step - loss: 1.2051 - accuracy: 0.4375
Epoch 2/5
1/1 [==============================] - 0s 1ms/step - loss: 1.2051 - accuracy: 0.4375
Epoch 3/5
1/1 [==============================] - 0s 996us/step - loss: 1.2051 - accuracy: 0.4375
Epoch 4/5
1/1 [==============================] - 0s 2ms/step - loss: 1.2051 - accuracy: 0.4375
Epoch 5/5


KeyboardInterrupt: 

In [92]:
test_results = my_model.evaluate(batchXt, batchyt, verbose=1)

1/1 [==============================] - 0s 1ms/step - loss: 0.6938 - accuracy: 0.5938


In [93]:
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

Test results - Loss: 0.6938470602035522 - Accuracy: 59.375%


In [107]:
batchX[0][0]

array([[  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  2.,   0.,   1.],
       [ 24.,   4.,   3.],
       [189.,  94.,  26.],
       [  2.,   0.,  12.],
       [  6.,   6.,   0.],
       [  0.,   0.,   4.],
       [  1.,   1.,   1.],
       [ 13.,   0.,   0.],
       [ 15.,   3.,   3.],
       [ 14.,   5.,   6.],
       [ 25.,  11.,  11.],
       [ 14.,  11.,   4.],
       [  9.,   8.,   4.],
       [ 15.,   3.,  15.],
       [227., 231., 243.],
       [240., 214., 227.],
       [251., 204., 212.],
       [255., 228., 239.],
       [ 84.,  38.,  12.],
       [103.,  56.,  28.],
       [117.,  57.,  33.],
       [114.,  77.,  69.],
       [144., 109.,  89.],
       [142., 117.,  97.],
       [139., 123., 100.],
       [ 86.,  34.,  20.],
       [109.,  47.,  24.],
       [  0.,   2.,   3.],
       [ 93.,  80.,  63.],
       [104.,  89.,  68.],
       [132., 121., 101.],
       [ 83.,  51.,  40.],
       [115.,  77.,  54.],
       [121.,  79.,  63.],
       [ 72.,  57.,  34.],
 

The model achieves less accuracy than the LDA from the pre-class work. I assume that I have some error in training the model. Unfortunately, I was not able to figure out where exactly the errors lies.